In [134]:
#from util import *
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
import pandas as pd
import matplotlib.pyplot as plt
from tqdm import tqdm

In [135]:
import scipy.io as scio
from scipy import signal
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

data_path = "/Users/n.y.r/Desktop/bcidata/datasets_matlab/"
electrodeID_path = '/Users/n.y.r/Desktop/BCI_Project/metadata/' #path to git repo CSV files

all_data = {}
all_data['N'] = {i: scio.loadmat(data_path + 'i140703-001_raw-lfp-spikes_ch{}.mat'.format(str(i).zfill(2))) for i in range(1,97)}
all_data['L'] = {i: scio.loadmat(data_path + 'l101210-001_raw-lfp-spikes_ch{}.mat'.format(str(i).zfill(2))) for i in range(1,97)}

def get_trial_time_and_label(monkey, event_want='GO-ON'):
    assert monkey in ['L', 'N']
    data = all_data[monkey][1]
    event_data = data['block']['segments'][0][0][0][0][0][0]['events'][0][2][0][0]
    assert event_data['name'] == 'TrialEvents'
    assert event_data['times_units'] == '(1.0/30000 * s)'
    samples = []
    t_trial = None
    lb_trial = None
    valid_trial = True
    valid_trial2 = False
    for event, label, time in zip(event_data['an_trial_event_labels'],
                            event_data['an_belongs_to_trialtype'],
                            event_data['times'][0]):
        if event.strip() == event_want.strip():
            t_trial = time/30000
            lb_trial = label
        if 'ERROR' in event:
            valid_trial = False
        if 'RW-ON' in event:
            valid_trial2 = True
        if event[:5] in ['TS-ON', 'NONE ']:
            if valid_trial and valid_trial2 and lb_trial:
                samples.append((t_trial, lb_trial))
            t_trial = None
            lb_trial = None
            valid_trial = True
    return samples

def get_electrodes(monkey_name, x, y, step):
    chunk_channel_id=[]
    assert monkey_name in ['L', 'N']
    if monkey_name=='L':
        df=pd.read_csv(electrodeID_path+'L.csv')
    else:
        df=pd.read_csv(electrodeID_path+'N.csv')   
    monkey_electrodes_id=df['Id'].tolist()
    start=(y-1)*10+x
    for col in range (0, step):
        for row in range (0,step):
            chunk_channel_id.append(monkey_electrodes_id[start+10*row-1])
        start=start+1   
    return chunk_channel_id

def get_st_feat(monkey, ch, start_time, window_time, unit_id_flag):
    data = all_data[monkey][ch]
    tmp = data['block']['segments'][0][0][0][0][0][0]['spiketrains'][0]
    st = None
    if unit_id_flag=='MIN':
        #print('1111111111111111111111')
        for i in range(tmp.shape[0]):
            if tmp[i][0][0]['an_unit_id'].item() == 1: st = tmp[i][0][0]
            elif tmp[i][0][0]['an_unit_id'].item() == 0 and st is None: st = tmp[i][0][0]
    if unit_id_flag=='MAX':
        #print('****************')
        if tmp[tmp.shape[0]-1][0][0]['an_unit_id'].item() != 255:
            st = tmp[tmp.shape[0]-1][0][0]
        else: 
            st = tmp[tmp.shape[0]-2][0][0]
    if unit_id_flag=='ZERO':
        st = tmp[0][0][0]
        
    assert st['times_units'].item() == '(1.0/30000 * s)'
    st['times'][0]
    st_op = start_time*30000
    st_ed = (start_time+window_time)*30000

    frs = []
    parts = 10
    for i in range(parts):
        sop = st_op + (st_ed-st_op)*i/parts
        sed = st_op + (st_ed-st_op)*(i+1)/parts
        frs.append(((st['times'][0] >= sop) & (st['times'][0] < sed)).sum())
    # return [((st['times'][0] >= st_op) & (st['times'][0] < st_ed)).sum()]
    return frs

def get_lfp_feat(monkey, ch, start_time, window_time):
    data = all_data[monkey][ch]
    lfp_data = data['block']['segments'][0][0][0][0][0][0]['analogsignals'][0][0][0][0]#['file_origin'][0][-3:]
    assert lfp_data['file_origin'][0][-3:] == 'ns2'
    assert lfp_data['t_start_units'] == '(1.0/1000.0*s)'
    start_time_shifted = start_time - lfp_data['t_start'].item()
    
    
    psds = []
    parts = 10
    for i in range(parts):
        op = int((start_time_shifted+window_time*i/parts)*1000)
        ed = int((start_time_shifted+window_time*(i+1)/parts)*1000)
        sig = lfp_data['signal'].squeeze()[op:ed]
        freqs, psd = signal.welch(sig, 1000) #nperseg=500)
        psds.append(psd[:])
    return np.concatenate(psds)

def get_all_feat_and_labels(monkey, event_want, offset, window_time,unit_id_flag):
    trials = get_trial_time_and_label(monkey, event_want)

    all_feat = {}

    for ch in range(1,97):
        st_feat = []
        if monkey == 'N': lfp_feat = []
        for trial_t, label in trials:
            start_time = trial_t + offset
            st_feat.append(get_st_feat(monkey, ch, start_time, window_time,unit_id_flag))
            if monkey == 'N': lfp_feat.append(get_lfp_feat(monkey, ch, start_time, window_time))
        all_feat[ch] = {'st_feat': np.stack(st_feat)}
        if monkey == 'N': all_feat[ch]['lfp_feat'] = np.stack(lfp_feat)
    labels = np.stack([trial[1] for trial in trials])
    
    return all_feat, labels

def plot_heatmap(accs, vmin=0.25, vmax=1):
    cols, rows = accs.shape
    plt.imshow(accs, interpolation='nearest', extent=[0.5, 0.5+cols, 0.5, 0.5+rows],
                      cmap='bwr', vmin=vmin, vmax=vmax)
    plt.colorbar()
    plt.axis('off')
    plt.show()

In [136]:
def run_single_time_window(monkey, event_want, offset, time_window, unit_id_flag,  spatial_step=3):
    spatial_xid=[]
    spatial_yid=[]
    acc_ls=[]
    time_id=[]
    all_feat, labels = get_all_feat_and_labels(monkey, event_want, offset, time_window,unit_id_flag)
    #labels = labels.view('<U2')[::2]
    for i in range (1, 10-spatial_step+2):
        for j in range (1, 10-spatial_step+2): 
            elecs = get_electrodes(monkey, i, j, spatial_step)
            data = []
            for ch in elecs:
                if ch == -1: continue
                data.append(all_feat[ch]['st_feat'])
            if monkey == 'N':
                for ch in elecs:
                    if ch == -1: continue
                    data.append(all_feat[ch]['lfp_feat'])
            data = np.concatenate(data, axis=1)
            train_num=int(0.8*data.shape[0])

            X_train, y_train, X_test, y_test = data[:train_num], labels[:train_num], data[train_num:], labels[train_num:]
            # Normalize data
            scaler = StandardScaler()  # normalization: zero mean, unit variance
            scaler.fit(X_train)  # scaling factor determined from the training set

            X_train = scaler.transform(X_train)
            X_test = scaler.transform(X_test)
            
           # pca = PCA(n_components=100)
           # pca.fit(X_train)
          #  X_train = pca.transform(X_train)
            #X_test = pca.transform(X_test)
            
            
#             clf = MLPClassifier(hidden_layer_sizes=(50), learning_rate_init=0.01, max_iter=500, verbose=0)
            clf = SVC(kernel='sigmoid', C=1)
            clf.fit(X_train, y_train)

            accuracy = (clf.predict(X_test) == y_test).mean()
            spatial_xid.append(i)
            spatial_yid.append(j)
            acc_ls.append(accuracy)
            time_id.append(offset)
    return pd.DataFrame(list(zip(spatial_xid, spatial_yid, time_id, acc_ls)), columns = ['X', 'Y', 'Time Stamp', 'Accuracy'])

In [154]:
accs = []
for _ in tqdm(range(10)):
    df = run_single_time_window('L', 'SR', -0.8, 1.6,'ZERO')
    accs.append(df['Accuracy'].mean())
np.mean(accs)

  0%|          | 0/10 [00:00<?, ?it/s]/Users/n.y.r/anaconda3/lib/python3.7/site-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/Users/n.y.r/anaconda3/lib/python3.7/site-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/Users/n.y.r/anaconda3/lib/python3.7/site-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/Users/n.y.r/anac

/Users/n.y.r/anaconda3/lib/python3.7/site-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/Users/n.y.r/anaconda3/lib/python3.7/site-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/Users/n.y.r/anaconda3/lib/python3.7/site-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/Users/n.y.r/anaconda3/lib/python3.7/site-packages/skle

/Users/n.y.r/anaconda3/lib/python3.7/site-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/Users/n.y.r/anaconda3/lib/python3.7/site-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/Users/n.y.r/anaconda3/lib/python3.7/site-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/Users/n.y.r/anaconda3/lib/python3.7/site-packages/skle

/Users/n.y.r/anaconda3/lib/python3.7/site-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/Users/n.y.r/anaconda3/lib/python3.7/site-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/Users/n.y.r/anaconda3/lib/python3.7/site-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/Users/n.y.r/anaconda3/lib/python3.7/site-packages/skle

/Users/n.y.r/anaconda3/lib/python3.7/site-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/Users/n.y.r/anaconda3/lib/python3.7/site-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/Users/n.y.r/anaconda3/lib/python3.7/site-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/Users/n.y.r/anaconda3/lib/python3.7/site-packages/skle

...

/Users/n.y.r/anaconda3/lib/python3.7/site-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/Users/n.y.r/anaconda3/lib/python3.7/site-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/Users/n.y.r/anaconda3/lib/python3.7/site-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/Users/n.y.r/anaconda3/lib/python3.7/site-packages/skle

/Users/n.y.r/anaconda3/lib/python3.7/site-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/Users/n.y.r/anaconda3/lib/python3.7/site-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/Users/n.y.r/anaconda3/lib/python3.7/site-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/Users/n.y.r/anaconda3/lib/python3.7/site-packages/skle

/Users/n.y.r/anaconda3/lib/python3.7/site-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/Users/n.y.r/anaconda3/lib/python3.7/site-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/Users/n.y.r/anaconda3/lib/python3.7/site-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/Users/n.y.r/anaconda3/lib/python3.7/site-packages/skle

/Users/n.y.r/anaconda3/lib/python3.7/site-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/Users/n.y.r/anaconda3/lib/python3.7/site-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/Users/n.y.r/anaconda3/lib/python3.7/site-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/Users/n.y.r/anaconda3/lib/python3.7/site-packages/skle

0.46145833333333314

In [155]:
np.mean(accs)

0.46145833333333314

In [148]:
accs

[]

In [149]:
df['Accuracy'].mean()

0.4614583333333332